In [46]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

# read in all files
train2015_05_28 = pd.read_csv("../input/train2015_05_28_changed.csv")
train2015_06_28 = pd.read_csv("../input/train2015_06_28_changed.csv")
train2016_05_28 = pd.read_csv("../input/train2016_05_28.csv")
testX = pd.read_csv("../input/testX.csv")

print(train2015_05_28.shape)
print(train2015_06_28.shape)
print(train2016_05_28.shape)
print(testX.shape)

# the unique ones in either 2015-05-28 or 2015-06-28
train2015_05_28_unique = pd.read_csv('../input/train2015_05_28_unique.csv')
print("Unique ones are: ", train2015_05_28_unique.shape)

train2015_06_28_unique = pd.read_csv('../input/train2015_06_28_unique.csv')
print("Unique ones are: ", train2015_06_28_unique.shape)

# read in changed id list
#changed_ids = []
#with open('changed_ids.txt', 'r') as f:
#    for line in f:
#        changed_ids.append(int(line.strip()))
#print("changed list is: ", len(changed_ids))

(100, 47)
(100, 47)
(100, 47)
(100, 23)
Unique ones are:  (3354, 47)
Unique ones are:  (3507, 47)


#### only 22 products has data. Products 'ind_ahor_fin_ult1' un 'ind_aval_fin_ult1' have not been purchased.
- so we need to drop these two columns
- in addition, the date information should also be removed!

In [24]:
train2015_05_28.drop(['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'fecha_dato', 'fecha_alta', 'ult_fec_cli_1t'], 1, inplace=True)
train2015_06_28.drop(['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'fecha_dato', 'fecha_alta', 'ult_fec_cli_1t'], 1, inplace=True)
train2016_05_28.drop(['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'fecha_dato', 'fecha_alta', 'ult_fec_cli_1t'], 1, inplace=True)
print(train2015_05_28.shape)
print(train2015_06_28.shape)
print(train2016_05_28.shape)

train2015_05_28_unique.drop(['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'fecha_dato', 'fecha_alta', 'ult_fec_cli_1t'], 1, inplace=True)
train2015_06_28_unique.drop(['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'fecha_dato', 'fecha_alta', 'ult_fec_cli_1t'], 1, inplace=True)
print("Unique ones are: ", train2015_05_28_unique.shape)
print("Unique ones are: ", train2015_06_28_unique.shape)

    ncodpers  age  ind_nuevo  antiguedad  indrel  indrel_1mes  tipodom  \
99   1065933   33        0.0        34.0     1.0            1      1.0   

    cod_prov  ind_actividad_cliente          renta       ...         \
99      27.0                    1.0  178865.279517       ...          

    ind_hip_fin_ult1  ind_plan_fin_ult1  ind_pres_fin_ult1  ind_reca_fin_ult1  \
99                 0                  0                  0                  0   

    ind_tjcr_fin_ult1  ind_valo_fin_ult1  ind_viv_fin_ult1  ind_nomina_ult1  \
99                  0                  0                 0              1.0   

    ind_nom_pens_ult1  ind_recibo_ult1  
99                1.0                1  

[1 rows x 47 columns]


#### For padding
- when compare two different months, some columns values are missing as they are unique to individual months
- in order to do the comparison, the month with missing customers will have to pad everything in.

In [26]:
target_cols = ['ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1',
               'ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1',
               'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1',
               'ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1',
               'ind_nom_pens_ult1','ind_recibo_ult1']

# for padding
padding_for_55 = train2015_06_28_unique.copy()
padding_for_55[target_cols] = [0] * 22

padding_for_56 = train2015_05_28_unique.copy()
padding_for_56[target_cols] = [0] * 22


print("padding for 55 is: ", padding_for_55.shape)
#print(train2015_06_28_unique[target_cols].iloc[1:2,:])
#print(padding_for_55[target_cols].iloc[1:2,:])

# compose the changed list for 2015-05-28
#train55 = pd.concat([train2015_05_28_unique])
train55 = pd.concat([train2015_05_28, train2015_05_28_unique, padding_for_55])
print("2015-05-28 only: ", train55.shape)

# compose the changed list for 2015-06-28
train56 = pd.concat([train2015_06_28, train2015_06_28_unique, padding_for_56])
#train56 = pd.concat([padding_for_56])
print("2015-06-28 only: ", train56.shape)

# changed list for all 2015
trainX2015 = pd.concat([train2015_05_28, train2015_05_28_unique, train2015_06_28_unique])
print("the entire list for 2015: ", trainX2015.shape)

padding for 55 is:  (3507, 45)
2015-05-28 only:  (100, 45)
2015-06-28 only:  (100, 45)
the entire list for 2015:  (6961, 45)


In [63]:
# now need to find out changed target in a single column list format

trainY = []
trainX = []

for idx, row in train56.iterrows():
    customer_id = row.ncodpers
    row55 = train55[train55.ncodpers == customer_id]        
    cur_target_list  = row[target_cols].values
    prev_target_list = row55[target_cols].values[0]
    row55.drop(['ncodpers'], 1, inplace=True)
    row55 = row55.values[0]
    #print(prev_target_list, " and ", cur_target_list)

    new_target_list =[max(abs(int(x1) - int(x2)), 0) for (x1, x2) in zip(cur_target_list, prev_target_list)]
    
    for ind, val in enumerate(new_target_list):
        if val != 0:
            trainX.append(row55)
            trainY.append(ind)
            
print(len(trainY))
print(len(trainX))

(100, 45)
[21, 19, 20, 19, 20, 19, 20, 2, 0, 2, 16, 21, 0, 21, 16, 0, 2, 16, 21, 19, 20, 21, 15, 21, 15, 16, 0, 10, 19, 20, 21, 19, 20, 21, 0, 2, 21, 10, 19, 20, 0, 2, 15, 21, 4, 10, 15, 19, 20, 20, 21, 21, 16, 19, 20, 0, 2]


In [ ]:
from collections import Counter
print(Counter(trainY))
#with open('trainY.txt', 'w') as f:
#    for i in trainY:
#        f.write("%d\n" % i)

#trainX = np.array(trainX)
#trainY = np.array(trainY)
print(trainY.shape)
print(trainX.shape)
print(trainX[0])

#trainY.dump("trainY.dat")
#trainX.dump("trainX.dat")

In [ ]:
import pandas as pd
import numpy as np

# now need to convert testX file
testX = pd.read_csv("../input/testX.csv")#, nrows=5)
print(testX.shape)

#testX[target_cols] = [0]*22
testX.drop(['ncodpers', 'fecha_dato', 'fecha_alta', 'ult_fec_cli_1t'], 1, inplace=True)

print(testX.shape)

testData = []
for idx, row in testX.iterrows():
    row = row.tolist()
    testData.append(row + [0]*22)
    
print(len(testData))

testX = np.array(testData)
print(testX.shape)
#print(testX)

#np.save("testX", testX)
#testX.dump("testX.dat")

In [37]:
import xgboost as xgb

params = {'seed': 0,
          'colsample_bytree': 0.7,
          'silent': 1,
          'subsample': 0.5,
          'learning_rate': 0.1,
          'objective': 'binary:logistic',
          'max_depth': 10,
          'min_child_weight': 100,
          'booster': 'gbtree', 
          'eval_metric': 'mlogloss',
         }

#params = {'seed': 125,
#          'colsample_bytree': 0.7,
#          'silent': 1,
#          'subsample': 0.7,
#          'eta': 0.05,
#          'objective': 'multi:softprob',
#          'max_depth': 8,
#          'min_child_weight': 1,
#          'eval_metric': 'mlogloss',
#          'num_class' : 22
#         }

num_rounds = 50

trainX = xgb.DMatrix(trainX, label=trainY)
testX = xgb.DMatrix(testX)
